# Capital asset pricing model (CAPM)

## Types of risk

Systematic and unsystematic risk are two types of risk that investors face when they make decisions in the financial markets. 

**Systematic risk**, is the risk that is inherent in the whole market or economy, this risk is faced by all financial instruments and cannot be eliminated using diversification. Examples of systematic risk are interest rate changes, political change, natural disasters, inflation and most recently a global pandemic.

**Unsystematic Risk**, is the risk that is particular to an individual security or company, this risk can be reduced or eliminated through diversification. Examples of unsystematic risk are management changes, product recalls and negative publicity.

## Capital asset pricing model formula

**Capital asset pricing model (CAPM) formula**

$$E(R_{i}) = R_f + \beta_{i}(E(R_m) - R_f)$$

Where:
* E(R_{i}) is the expected return of the security i
* R_f is the risk-free rate of return
* β_1 is the beta coefficient of the security
* E(R_{m}) is the expected return of the market portfolio

**β (Beta) coefficient formula**

$$\beta = \frac{cov(r_i, r_m)}{var(r_m)}$$

The β coefficient represents the systematic risk of a security or portfolio in comparison to the market as a whole. Beta measures the sensitivity of the portfolio or security to changes in the market returns. Beta values to consider:
* Beta values that are equal to one indicate the asset shares the same volatility as the market. 
* Beta values that are greater than one indicate that the asset is more volatile than the market.
* Beta values that are less than one indicate that the asset is less volatile than the market.

By selecting securities with betas that align with their risk tolerance and investment objectives, investors can build a diversified portfolio that balances risk and return.

## Python programming for the CAPM model

### Import the required libraries

This Python code imports several libraries for data analysis and visualization, which are commonly used in finance, economics, and other fields that deal with data analysis and modeling.
* numpy is a library used for mathematical operations.
* pandas is a library used for data manipulation and analysis. It will allow us to import the stock data into DataFrames and manipulate the data. 
* yfinance is a library used for retrieving financial data from Yahoo Finance, this will allow us to import the relevant stock and market data for the CAPM. 
* matplotlib.pyplot is a plotting library used for data visualization.
* statsmodels.api is a library used for statistical data analysis, from which we will use the linear regression model.

In [99]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm

### Capital Asset Pricing Model class

In [123]:
class CAPM():
    
    def __init__(self, stocks, start_date, end_date):
        self.data = None
        self.stocks = stocks
        self.start_date = start_date
        self.end_date = end_date
        
    def download_data(self):
        
        # Data dictionary with key as stock and value as price data
        data = {}
        
        for stock in self.stocks:
            if '^' in stock:
                ticker = yf.download(stock.replace('^', '-'), self.start_date, self.end_date)
            else:
                ticker = yf.download(stock, self.start_date, self.end_date)
            data[stock] = ticker['Adj Close']
        
        return pd.DataFrame(data)
    
    def initialize(self):
        
        stocks = self.download_data()
        
        stocks = stocks.resample('M').last() 
        
        self.data = pd.DataFrame({'s_adjclose':stocks[self.stocks[0]], 
                                  'm_adjclose':stocks[self.stocks[1]]})
        
        # logarithmic monthly returns
        self.data[['s_returns', 'm_returns']] = np.log((self.data[['s_adjclose', 'm_adjclose']] / 
                                                     self.data[['s_adjclose', 'm_adjclose']].shift(1)))
        
        # Removes NaN values from zero division
        self.data = self.data[1:]
        
        return self.data
    
    def calculate_beta(self, method='linear_regression', volatility_status=True):
        # Loop for method type
        if method == "covariance_approach":
            cov_matrix = np.cov(self.data['s_returns'], self.data['m_returns'])
            market_var = np.var(self.data['m_returns'])
            
            beta = cov_matrix[0, 1] / market_var
        elif method == "linear_regression":
            # Linear regression - sm fit
            X = sm.add_constant(self.data['m_returns'])
            model = sm.OLS(self.data['s_returns'], X)
            results = model.fit()
            
            # Model parameters
            alpha = results.params[0]
            beta = results.params[1]
            
            print('Alpha coefficient: ', alpha)
        else:
            raise ValueError("Invalid method variable name. Choose either 'covariance_approach' or 'linear_regression'")
        
        # Loop for average
        if volatility_status == True:
            if beta > 1:
                volatility = 'Higher than average volatility'
            elif beta == 1:
                volatility = 'Similar volatility'
            else:
                volatility = 'Below average volatility'
            print('Beta coefficient: ', beta)
            print('Compared to Market: ', volatility)
        elif volatility_status == False:
            print('Beta coefficient: ', beta)
        else:
            raise ValueError("Invalid volatility_status. Choose either True or False")

In [124]:
stock_and_market = ['AMZN', 'TSLA']
camp = CAPM(stock_and_market, '2015-01-01', '2019-01-01')

In [125]:
camp.initialize()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,s_adjclose,m_adjclose,s_returns,m_returns
Date,,,,
2015-02-28,19.007999,13.556000,0.069799,-0.001278
2015-03-31,18.605000,12.584667,-0.021430,-0.074350
2015-04-30,21.089001,15.070000,0.125321,0.180227
2015-05-31,21.461500,16.719999,0.017509,0.103900
2015-06-30,21.704500,17.884001,0.011259,0.067301
2015-07-31,26.807501,17.743334,0.211162,-0.007897
2015-08-31,25.644501,16.604000,-0.044353,-0.066366
2015-09-30,25.594500,16.559999,-0.001952,-0.002654
2015-10-31,31.295000,13.795333,0.201081,-0.182660


In [126]:
camp.calculate_beta(method='covariance_approach')

Beta coefficient:  0.09696745367339782
Compared to Market:  Below average volatility


In [127]:
camp.calculate_beta()

Alpha coefficient:  0.029725647885636884
Beta coefficient:  0.09490431636119792
Compared to Market:  Below average volatility
